In [14]:
import pymysql
from dotenv import load_dotenv
import os

load_dotenv()

db = pymysql.connect(
    host=os.getenv('DATABASE_HOST'),
    port=int(os.getenv('DATABASE_PORT')),
    user=os.getenv('DATABASE_USER'),
    passwd=os.getenv('DATABASE_PASSWORD'),
    db=os.getenv('DATABASE_DATABASE'),
    charset='utf8'
    )

con = db.cursor()

In [15]:
sql = "SELECT * FROM food"
con.execute(sql)
results = con.fetchall()
for row in results:
    print(row)

In [16]:
import json

with open('./recipe1~1000.json', 'r') as file:
    data = json.load(file)

In [17]:
foodList = data["COOKRCP01"]["row"]
print(foodList[0])

{'RCP_PARTS_DTLS': '새우두부계란찜\n연두부 75g(3/4모), 칵테일새우 20g(5마리), 달걀 30g(1/2개), 생크림 13g(1큰술), 설탕 5g(1작은술), 무염버터 5g(1작은술)\n고명\n시금치 10g(3줄기)', 'RCP_WAY2': '찌기', 'MANUAL_IMG20': '', 'MANUAL20': '', 'RCP_SEQ': '28', 'INFO_NA': '99', 'INFO_WGT': '', 'INFO_PRO': '14', 'MANUAL_IMG13': '', 'MANUAL_IMG14': '', 'MANUAL_IMG15': '', 'MANUAL_IMG16': '', 'MANUAL_IMG10': '', 'MANUAL_IMG11': '', 'MANUAL_IMG12': '', 'MANUAL_IMG17': '', 'MANUAL_IMG18': '', 'MANUAL_IMG19': '', 'INFO_FAT': '17', 'HASH_TAG': '연두부', 'MANUAL_IMG02': 'http://www.foodsafetykorea.go.kr/uploadimg/cook/20_00028_2.png', 'MANUAL_IMG03': 'http://www.foodsafetykorea.go.kr/uploadimg/cook/20_00028_3.png', 'RCP_PAT2': '반찬', 'MANUAL_IMG04': '', 'MANUAL_IMG05': '', 'MANUAL_IMG01': 'http://www.foodsafetykorea.go.kr/uploadimg/cook/20_00028_1.png', 'MANUAL01': '1. 손질된 새우를 끓는 물에 데쳐 건진다.a', 'ATT_FILE_NO_MK': 'http://www.foodsafetykorea.go.kr/uploadimg/cook/10_00028_1.png', 'MANUAL_IMG06': '', 'MANUAL_IMG07': '', 'MANUAL_IMG08': '', 'MANUAL_IMG09': '

In [18]:
insertFoodSql = "INSERT INTO food(id, name, main_image, description, type, ingredient, like_count, view_count, calorie, carbohydrate, protein, province, salt) VALUES(%s,%s,%s,%s,%s,%s,0,0,%s,%s,%s,%s,%s)"

data = []
for i in range(0, 1000):
    data.append((int(foodList[i]['RCP_SEQ']), 
                 foodList[i]['RCP_NM'],
                 foodList[i]['ATT_FILE_NO_MAIN'],
                 foodList[i]['RCP_NA_TIP'],
                 foodList[i]['RCP_WAY2'],
                 foodList[i]['RCP_PARTS_DTLS'],
                 int(float(foodList[i]['INFO_ENG'])), 
                 int(float(foodList[i]['INFO_CAR'])),
                 int(float(foodList[i]['INFO_PRO'])),
                 int(float(foodList[i]['INFO_FAT'])),
                 int(float(foodList[i]['INFO_NA']))))

try:
    con.executemany(insertFoodSql, data)  # executemany로 데이터 삽입
    db.commit()  # 모든 데이터 삽입 후에 커밋
except Exception as e:
    print(e)
    db.rollback()  # 롤백하여 이전 상태로 복원


In [19]:
insertRecipeSql = "INSERT INTO recipe(id, food, recipeOrder, image, description) VALUES(%s,%s,%s,%s,%s)"
autoIcrId = 1

data = []
for i in range(0, 1000):
    manualIndex = 1
    while(foodList[i]['MANUAL' + str(manualIndex).zfill(2)] != ""):
        data.append((autoIcrId, 
                     int(foodList[i]['RCP_SEQ']),
                     manualIndex,
                     foodList[i]['MANUAL_IMG' + str(manualIndex).zfill(2)],
                     foodList[i]['MANUAL' + str(manualIndex).zfill(2)]))
        autoIcrId += 1
        manualIndex += 1

try:
    con.executemany(insertRecipeSql, data)  # executemany로 데이터 삽입
    db.commit()  # 모든 데이터 삽입 후에 커밋
except Exception as e:
    print(e)
    db.rollback()  # 롤백하여 이전 상태로 복원